In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import json
import time
from anytree import Node, RenderTree
import time
from PttWebCrawler.crawler import *


In [8]:
def get_urls_by_url(board,url):
    base_url="https://www.ptt.cc/man/"+board+"/"+url
    r = requests.get(base_url)
    if r.status_code == requests.codes.ok:
        soup = BeautifulSoup(r.text, 'html.parser')
        links=soup.find_all('a')
        urls=[x.get('href') for x in links]
        return urls

In [53]:
def get_article_ids(root_url,board):
    #Here we define index url as "link" eg:https://www.ptt.cc/man/marvel/DFB6/D6BD/index.html
    #article url as "article" eg:https://www.ptt.cc/man/marvel/DFB6/D6BD/M.1477757823.A.549.html
    layer1=[root_url]
    layer_list=[layer1]
    i=0
    while True:
        layer_list+=[[]]
        print("retreving layer",i+1)
        current_layer=layer_list[i]
        next_layer=layer_list[i+1]

        current_links=[x for x in current_layer if x.endswith("/index.html")]
        for current_link_url in current_links:
            next_urls=[]
            for attempt in range(10):
                try:
                    next_urls=get_urls_by_url(board,current_link_url)
                except TypeError:
                    time.sleep(1)
                    print("error occur in link:",current_link_url, " ,try again")
                    continue
                if next_urls==None:
                    time.sleep(1)
                    print("error occur in link:",current_link_url, " ,try again")
                    continue
                else:
                    break
            # leave urls end with "M.xxxxx" and "xxx/xxx/index.html"
            next_urls=[x for x in next_urls if x.startswith("/man/"+board+"/") and not x.endswith(current_link_url) and x!="/man/"+board+"/index.html"]
            next_layer+=[x.replace("/man/"+board+"/","") for x in next_urls]
        if i>0:
            pre_layer=layer_list[i-1]
            layer_list[i+1]=[x for x in next_layer if x not in pre_layer]
        have_next_layer=any([x for x in layer_list[i+1] if x.endswith("/index.html")])
        i+=1
        print("total links: ",len(layer_list[i]))
        if have_next_layer is False:
            print("layer",str(i)," is the last layer")
            break
    #get all article (links end with M.xxxx)
    all_article=[]
    for layer in layer_list:
        if len(layer) >0:
            all_article+=[x for x in layer if not x.endswith("index.html")]
    all_index_page=[]        
    for layer in layer_list:
        if len(layer) >0:
            all_index_page+=[x for x in layer if x.endswith("index.html")]
    #remove ".html"
    all_article=[x[:-5] for x in all_article]
    write_list_to_txt(board+"-"+root_url,all_article)
    print("--------")
    print("Total articles:",len(all_article))
    print("Total index page:",len(all_index_page))
    return all_article
    

In [54]:
def write_list_to_txt(file_name,article_list):
    file_name=file_name.replace("/","-")
    with open(file_name+".txt", 'w') as f:
        for item in article_list:
            f.write("%s\n" % item)

In [55]:
def crawling_data_by_ids(board,id_list):
    c = PttWebCrawler(as_lib=True)
    count=0
    for target in id_list:
        try:
            c.parse_article(target, board)
        except Exception as e:
            print(e)
            continue
        count+=1
        if count%100==0:
            print("progress:", count, "/", len(article_id_list))

In [56]:
# type in the root url eg: "DDF/index.html"
root_url="DDF/index.html"
# type in the board name eg: "marvel"
board="marvel"

In [57]:
article_id_list=get_article_ids(root_url,board)

retreving layer 1
total links:  2
retreving layer 2
total links:  3
retreving layer 3
total links:  8
layer 3  is the last layer
--------
Total articles: 8
Total index page: 6


In [58]:
crawling_data_by_ids(board,article_id_list)

Processing article: DDF-DD90-DDAA-M.1445699949.A.994
Processing article: DDF-DD90-DDAA-M.1445859799.A.E46
Processing article: DDF-DD90-DDAA-M.1445859806.A.3E6
Processing article: DDF-DDB4-DDC6-M.1448103380.A.624
Processing article: DDF-DDB4-DDC6-M.1448108011.A.3B7
Processing article: DDF-DDB4-DDC6-M.1448108020.A.A5C
Processing article: DDF-DDB4-DDC6-M.1448157088.A.6B8
Processing article: DDF-DDB4-DDC6-M.1448157097.A.B16
